# **Phase 2: Data Preparation**
- **Goal**: This notebook makes neccessary preparation for the data to be used further in this project


In [1]:
# import dependencies
import numpy as np
import pandas as pd
import glob
import os

In [2]:
# Load data from multiple CSV files in a directory, excluding a specific file, and merge them into a single DataFrame.

# Path to the data folder directory
# data_path = "data/"  
data_path = "C:/Nam/data_prep-prj/Data" 

# Get all CSV files in the specified directory
all_csv_files = glob.glob(os.path.join(data_path, "*.csv"))

# Initialize an empty list to hold individual DataFrames
list_of_dfs = []

for file_path in all_csv_files:
    filename = os.path.basename(file_path)
    
    # Skip News_Final.csv
    if filename == "News_Final.csv":
        print(f"[!] Skip file {filename}.")
        continue
        
    # Read each CSV file into a DataFrame
    try:
        df_temp = pd.read_csv(file_path)
        
        # Extract Source and Topic from the filename
        name_only = filename.replace('.csv', '') 
        parts = name_only.split('_') 
        
        if len(parts) == 2:
            # Add 'Source' and 'Topic' columns
            df_temp['Platform'] = parts[0]
            df_temp['Topic'] = parts[1]
            list_of_dfs.append(df_temp)
        else:
            print(f"[!] Skip file {filename} because it doesn't have 'Source_Topic.csv'")
            
    except Exception as e:
        print(f"[ERROR] Failed to read file {file_path}: {e}")

# Concatenate all DataFrames into a single DataFrame
if list_of_dfs:
    df = pd.concat(list_of_dfs, ignore_index=True)
    print(f"\n Successfully merged {len(list_of_dfs)} file.")
    print(f"DataFrame includes: {df.shape[0]} rows và {df.shape[1]} columns.")
    
    # Display the first few rows of the merged DataFrame
    print(df.head())
else:
    print("\n No files were merged.")

[!] Skip file News_Final.csv.

 Successfully merged 12 file.
DataFrame includes: 260515 rows và 147 columns.
   IDLink  TS1  TS2  TS3  TS4  TS5  TS6  TS7  TS8  TS9  ...  TS137  TS138  \
0     1.0   -1   -1   -1   -1   -1   -1   -1   -1    7  ...     13     13   
1     2.0   -1   -1   -1   -1   -1   -1   -1   -1    3  ...     42     42   
2     3.0   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...     98     98   
3     4.0   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...      7      7   
4     5.0   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...     34     34   

   TS139  TS140  TS141  TS142  TS143  TS144  Platform    Topic  
0     13     13     13     13     13     13  Facebook  Economy  
1     42     42     42     42     42     42  Facebook  Economy  
2     98     98     98     98     98     98  Facebook  Economy  
3      7      7      7      7      7      7  Facebook  Economy  
4     34     34     34     34     34     34  Facebook  Economy  

[5 rows x 147 columns]


In [3]:
# sort DataFrame by 'IDLink' and reset index
df = df.sort_values(by=['IDLink']).reset_index(drop=True)

In [4]:
# Reshape wide TS columns to long format (one row per article × time × platform)

# Identify TS columns (re-compute to be safe)
ts_columns = [col for col in df.columns if col.startswith('TS')]

id_vars = ['IDLink', 'Platform', 'Topic']
missing = [c for c in id_vars if c not in df.columns]
if missing:
    raise KeyError(f"Missing required id_vars in df: {missing}")

# Melt to long format
df_long = pd.melt(df,
                  id_vars=id_vars,
                  value_vars=ts_columns,
                  var_name='TimeSlice',
                  value_name='Popularity')

In [5]:
# Load metadata (News_Final.csv) 
meta_data = os.path.join(data_path, "News_Final.csv")
try:
    df_meta = pd.read_csv(meta_data)
except Exception as e:
    raise FileNotFoundError(f"Failed to load News_Final.csv: {e}")

In [6]:
# Create master dataset by merging df_long with df_meta
master_df = pd.merge(df_long, df_meta, how='outer', on='IDLink')

In [7]:
# Convert data types
master_df['IDLink'] = master_df['IDLink'].astype(str).str.replace('.0', '')  # Remove decimals
master_df['Popularity'] = master_df['Popularity'].astype(int, errors='ignore')  # Convert to integer

In [8]:
# Drop duplicate column: Topic_y
master_df = master_df.drop(columns=['Topic_y'])

# Rename columns for clarity
master_df = master_df.rename(columns={'Topic_x': 'Topic'})
master_df = master_df.rename(columns={'Popularity': 'Popularity_at_TimeSlice'})

In [9]:
# Replace -1 with NaN
master_df = master_df.replace(-1, np.nan)

In [10]:
# Check for unique articles
master_df['IDLink'].nunique()

93235

In [11]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37520015 entries, 0 to 37520014
Data columns (total 14 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   IDLink                   object 
 1   Platform                 object 
 2   Topic                    object 
 3   TimeSlice                object 
 4   Popularity_at_TimeSlice  float64
 5   Title                    object 
 6   Headline                 object 
 7   Source                   object 
 8   PublishDate              object 
 9   SentimentTitle           float64
 10  SentimentHeadline        float64
 11  Facebook                 float64
 12  GooglePlus               float64
 13  LinkedIn                 float64
dtypes: float64(6), object(8)
memory usage: 3.9+ GB


In [12]:
master_df

,IDLink,Platform,Topic,TimeSlice,Popularity_at_TimeSlice,Title,Headline,Source,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,1,Facebook,Economy,TS1,NaN,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,2015-11-09 22:40:10,0.087462,-0.093750,13.0,1.0,2.0
1,1,LinkedIn,Economy,TS1,NaN,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,2015-11-09 22:40:10,0.087462,-0.093750,13.0,1.0,2.0
2,1,GooglePlus,Economy,TS1,NaN,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,2015-11-09 22:40:10,0.087462,-0.093750,13.0,1.0,2.0
3,1,Facebook,Economy,TS2,NaN,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,2015-11-09 22:40:10,0.087462,-0.093750,13.0,1.0,2.0
4,1,LinkedIn,Economy,TS2,NaN,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,2015-11-09 22:40:10,0.087462,-0.093750,13.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37520010,104802,Facebook,Obama,TS143,0.0,"Obama leaves for NATO summit, visit to Spain",President Barack Obama left Thursday to take p...,AFP via Yahoo! News,2016-07-07 08:28:00,-0.099437,0.118648,0.0,0.0,0.0
37520011,104802,LinkedIn,Obama,TS143,0.0,"Obama leaves for NATO summit, visit to Spain",President Barack Obama left Thursday to take p...,AFP via Yahoo! News,2016-07-07 08:28:00,-0.099437,0.118648,0.0,0.0,0.0
37520012,104802,GooglePlus,Obama,TS144,0.0,"Obama leaves for NATO summit, visit to Spain",President Barack Obama left Thursday to take p...,AFP via Yahoo! News,2016-07-07 08:28:00,-0.099437,0.118648,0.0,0.0,0.0
37520013,104802,Facebook,Obama,TS144,0.0,"Obama leaves for NATO summit, visit to Spain",President Barack Obama left Thursday to take p...,AFP via Yahoo! News,2016-07-07 08:28:00,-0.099437,0.118648,0.0,0.0,0.0
